In [1]:
import pandas as pd
import numpy as np

data_loc = 'C:/Users/mattg/Downloads/'
file_name = 'nba_user_player_sentiment.tsv'


players2018.tsv
df = pd.read_csv(data_loc + file_name, sep='\t')
df['player_season'] = df['Player'] + df['season'].astype(str)
df['white'] = df['Race'] == 'W'
print(df.columns.values)

C:\Users\mattg\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['Player' 'user' 'flair' 'season' 'compound_mean' 'compound_std'
 'comment_count' 'user_Tm' 'Race' 'From' 'To' 'Pos_x' 'Ht' 'Wt'
 'Birth Date' 'Colleges' 'Rk' 'Pos_y' 'Age' 'G' 'MP' 'PER' 'TS%' '3PAr'
 'FTr' 'ORB%' 'DRB%' 'TRBP' 'ASTP' 'STLP' 'BLKP' 'TOVP' 'USG%' 'OWS' 'DWS'
 'WS' 'WS/48' 'OBPM' 'DBPM' 'BPM' 'VORP' 'Tm' 'FG' 'FGA' 'FG%' '3P' '3PA'
 'ThreePP' '2P' '2PA' '2P%' 'eFG%' 'FT' 'FTA' 'FT%' 'ORB' 'DRB' 'TRB'
 'AST' 'STL' 'BLK' 'TOV' 'PF' 'PPG' 'year' 'Wins' 'salary'
 'standard_salary' 'experience' 'user_total_score' 'user_total_comments'
 'user_mean_score' 'team_match' 'clinton_vote_lead' 'total_population'
 'metro_percent_white' 'metro_percent_black' 'youth' 'oldness' 'rookie'
 'white_black_diff' 'height_dummies' 'player_season' 'white']


In [2]:
#Need to drop null columns before pass to wls if clustering
cols_to_use = ['comment_count', 'white', 'compound_mean', 'user', 'Player', 'PPG', 'AST', 'DWS', 'BLK', 'TRB',  'Wins', 'TOV', 'user_total_comments',  'user_mean_score',  'user_total_score',   'season']
df_use = df.loc[:, cols_to_use].dropna(how='any')
print(len(df))
print(len(df_use))

713065
600075


In [3]:
#unclustered standard errors. Standard errors likely to be too small. Notice how white is significant here,
#but not in clustered regressions below

import statsmodels.formula.api as smf

formula = 'compound_mean ~ \
TOV + AST + DWS + BLK + TRB + PPG  + Wins + \
user_total_comments + user_mean_score + user_total_score + \
C(season) + white'

mod = smf.wls(formula,
              data=df_use,
              weights = (df_use['comment_count']),
             ).fit()
print(mod.summary())

                            WLS Regression Results                            
Dep. Variable:          compound_mean   R-squared:                       0.002
Model:                            WLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     72.88
Date:                Thu, 08 Nov 2018   Prob (F-statistic):          4.76e-194
Time:                        16:21:53   Log-Likelihood:            -1.8365e+05
No. Observations:              600075   AIC:                         3.673e+05
Df Residuals:                  600061   BIC:                         3.675e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.0936    

In [4]:
#the groups keyword is crucial. That is the column we "cluster at".
#Clustering is an important way to deal with the possibility of dpendence across a particular variable.
#It computes correct standard errors as if there is unrestricted dependence within vlaues of hte cluter column,
#but full indepdnence across different values of the cluster columns.
#Pick the right variable to cluster at is critical. Here we cluster at Player, below at user
#Note that clustering does not impact parmaeter estimates AT ALL but only corresponding standard errors
#More on clustering here: https://www.stata.com/support/faqs/statistics/standard-errors-and-vce-cluster-option/

mod = smf.wls(formula,
              data=df_use,
              weights = (df_use['comment_count']),
             ).fit(
    cov_type='cluster', cov_kwds={'groups' : df_use['Player']}
)
print(mod.summary())

                            WLS Regression Results                            
Dep. Variable:          compound_mean   R-squared:                       0.002
Model:                            WLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     17.21
Date:                Thu, 08 Nov 2018   Prob (F-statistic):           1.06e-34
Time:                        16:21:56   Log-Likelihood:            -1.8365e+05
No. Observations:              600075   AIC:                         3.673e+05
Df Residuals:                  600061   BIC:                         3.675e+05
Df Model:                          13                                         
Covariance Type:              cluster                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.0936    

In [5]:
#Cluster at user

mod = smf.wls(formula,
              data=df_use,
              weights = (df_use['comment_count']),
             ).fit(
    cov_type='cluster', cov_kwds={'groups' : df_use['user']}
)
print(mod.summary())

                            WLS Regression Results                            
Dep. Variable:          compound_mean   R-squared:                       0.002
Model:                            WLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     43.08
Date:                Thu, 08 Nov 2018   Prob (F-statistic):          7.59e-111
Time:                        16:22:00   Log-Likelihood:            -1.8365e+05
No. Observations:              600075   AIC:                         3.673e+05
Df Residuals:                  600061   BIC:                         3.675e+05
Df Model:                          13                                         
Covariance Type:              cluster                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.0936    